## ETL: Preparación de datasets

Extracción:  
En primera instancia obtendremos los datasets en crudo del siguiente repositorio: https://github.com/soyHenry/fe-ct-pimlops2  
O de la siguiente página: https://drive.google.com/drive/folders/1X_LdCoGTHJDbD28_dJTxaD4fVuQC9Wt5  
De estos links obtenremos dos archivos en formato .csv.  
Veremos paso a paso una guía breve para realizar una limpieza y obtener los datasets aptos para subir a un repositorio de github y que puedan ser consumidos por nuestra API a través de Render.
  
Una vez los archivos en nuestra computadora, procedemos a importar los archivos y a comenzar la transformación.  
  
Transformación:

In [2]:
import pandas as pd
import ast
import numpy as np
import json

# Cargar el CSV en un DataFrame
df_movies = pd.read_csv('Datasets/movies_dataset.csv')
df_credits =pd.read_csv('Datasets/credits.csv')

C:\Users\Usuario\AppData\Local\Temp\ipykernel_11364\1383607456.py:7: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies = pd.read_csv('C:/Users/Usuario/Desktop/Data Science/SoyHenry/PI1_files/Datasets/movies_dataset.csv')


In [2]:
#Visualizamos las dimensiones del dataframe movies, sus tipos, sus nulos y su peso.
print(df_movies.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [3]:
#Visualizamos las dimensiones del dataframe credits, sus tipos, sus nulos y su peso.
print(df_credits.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45476 entries, 0 to 45475
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   cast    45476 non-null  object
 1   crew    45476 non-null  object
 2   id      45476 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.0+ MB
None


In [4]:
df_credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


Tenemos una lista con objetivos que cumplir en esta etapa de desarrollo del proyecto:

1) Desanidado de columnas:  
Identificar qué columnas están anidadas: belongs_to_collection, production_companies, genres, production_countries y spoken language. También será necesario desanidar el dataset Credits.

2) Tratamiento de valores nulos:  
Nulos cambiados por cero: Revenue y Budget.
Nulos a eliminar: Release.

3) Tratamiento de fechas:  
Formato pedido: AAAA-mm-dd  
  
4) Creación de columnas nuevas:  
Creación de columna release_year: año de la fecha de estreno.  
Creación de columna return: Retorno de inversión; revenue / budget (si no hay datos disponibles para calcularlo, deberá tomar el valor 0)

5) Eliminación de columnas:  
Video, imdb_id, adult, original_title, poster_path y homepage.

In [5]:
# 1) DESANIDADO DE COLUMNAS:

# Probamos técnicas para desanidado con una sola columna en un principio.
column_name = 'genres'

# Definimos una función que maneje NaN y cadenas mal formateadas
def safe_literal_eval(val):
    try:
        if pd.isna(val):
            return []
        return ast.literal_eval(val)
    except (ValueError, SyntaxError):
        return []

# Aplicamos la función a la columna para convertirla a listas de diccionarios
df_movies[column_name] = df_movies[column_name].apply(safe_literal_eval)

# Extraemos las IDs y Names en nuevas columnas
df_movies['genres_id'] = df_movies[column_name].apply(lambda x: [d['id'] for d in x])
df_movies['genres_name'] = df_movies[column_name].apply(lambda x: [d['name'] for d in x])

# Eliminamos la columna original si ya no es necesaria
df_movies.drop(columns=['genres'], inplace=True)

# Guardamos el resultado en un nuevo archivo CSV
#df_movies.to_csv('movies_dataset_transformado_2.csv', index=False)

print("Archivo CSV con columnas expandidas guardado como 'movies_dataset_transformado_2.csv'")

Archivo CSV con columnas expandidas guardado como 'movies_dataset_transformado_2.csv'


In [7]:
# Repetimos el proceso con las otras columnas
columns_to_process = {
    'belongs_to_collection': ['id', 'name', 'poster_path', 'backdrop_path'],
    'production_companies': ['id', 'name'],
    'production_countries': ['iso_3166_1', 'name'],
    'spoken_languages': ['iso_639_1', 'name']
}

# Procesamos cada columna en la lista
for column_name, keys in columns_to_process.items():
    # Convertimos la columna a listas de diccionarios
    df_movies[column_name] = df_movies[column_name].apply(safe_literal_eval)

    # Verificamos si hay valores inesperados en la columna
    invalid_values = df_movies[~df_movies[column_name].apply(lambda x: isinstance(x, list))][column_name]
    if not invalid_values.empty:
        print(f"Valores inesperados en la columna {column_name}:")
        print(invalid_values)
    
    # Extraemos las claves correspondientes y crear nuevas columnas
    for key in keys:
        df_movies[f'{column_name}_{key.upper()}'] = df_movies[column_name].apply(lambda x: [d.get(key, np.nan) for d in x if isinstance(d, dict)])

# Guardamos el resultado en un nuevo archivo CSV
# df_movies.to_csv('movies_dataset_transformado_final.csv', index=False)

print("Archivo CSV con columnas expandidas guardado como 'movies_dataset_transformado_final.csv'")

C:\Users\Usuario\AppData\Local\Temp\ipykernel_3908\136982011.py:9: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if pd.isna(val):


Valores inesperados en la columna production_companies:
19730    False
29503    False
35587    False
Name: production_companies, dtype: object


TypeError: 'bool' object is not iterable

Hay columnas con valores booleanos! Esto no nos permitirá desarrollar el proceso de desanidado, siendo que son tres registros (contra 45K), procederemos a eliminarlos.

In [ ]:
# Lista de índices de las filas que queremos eliminar
indices_to_remove = [19730, 29503, 35587]

# Eliminamos las filas correspondientes
df_cleaned = df_movies.drop(indices_to_remove)

# ^^^^^^^^^^^^^^^^^^Repetimos el proceso del cuadro anterior!^^^^^^^^^^^^^^^^^^

Archivo CSV limpio guardado como 'movies_dataset_registros_borrrados.csv'


In [ ]:
# Eliminamos la columna original si ya no es necesaria
df_movies.drop(columns=['belongs_to_collection','production_companies', 'production_countries', 'spoken_languages'], inplace=True)

# Guardamos el DataFrame limpio en un nuevo archivo CSV
#df_cleaned.to_csv('movies_dataset_registros_borrrados.csv', index=False)

print("Archivo CSV limpio guardado como 'movies_dataset_registros_borrrados.csv'")

In [ ]:
# SEGUIMOS CON EL DATASET CREDITS:
# Lo separaremos en dos datasets; Crew y Cast.

# CREW
# Filtramos filas donde 'crew' tenga longitud mayor a 0
df_valid_crew = df_credits[df_credits['crew'].str.len() > 0]

# Convertimos la columna 'crew' a listas de diccionarios si es necesario
df_valid_crew['crew'] = df_valid_crew['crew'].apply(lambda x: eval(x) if isinstance(x, str) else x)

# Expandimos diccionarios en filas separadas
df_crew_expanded = df_valid_crew.explode('crew').reset_index(drop=True)

# Convertimos los diccionarios en columnas
df_crew_expanded = pd.json_normalize(df_crew_expanded['crew'])

# Verificamos la longitud antes de repetir 'id'
expected_length = df_crew_expanded.shape[0]
repeated_ids = df_valid_crew.loc[df_valid_crew.index.repeat(df_valid_crew['crew'].str.len()), 'id'].values

# Si la longitud no coincide, ajustar
if len(repeated_ids) > expected_length:
    repeated_ids = repeated_ids[:expected_length]
elif len(repeated_ids) < expected_length:
    df_crew_expanded = df_crew_expanded.iloc[:len(repeated_ids)]

# Asignamos la columna 'id'
df_crew_expanded['id'] = repeated_ids

# Restablecemos el índice
df_crew_expanded.reset_index(drop=True, inplace=True)

print(df_crew_expanded.head())

In [ ]:
# df_crew_expanded.to_csv('C:/Users/Usuario/Desktop/Data Science/SoyHenry/PI1/df_crew_expanded.csv', index=False)

In [ ]:
# CAST
# Filtramos filas con longitud mayor a 0
df_valid_cast = df_credits[df_credits['cast'].str.len() > 0]

# Expandimos diccionarios en filas separadas
df_cast_expanded = df_valid_cast.explode('cast')

# Convertimos los diccionarios en columnas
df_cast_expanded = pd.json_normalize(df_cast_expanded['cast'])

# Agregamos la columna 'id' del DataFrame original
df_cast_expanded['id'] = df_valid_cast.loc[df_valid_cast.index.repeat(df_valid_cast['cast'].str.len()), 'id'].values

# Restablecemos el índice
df_cast_expanded.reset_index(drop=True, inplace=True)

print(df_cast_expanded.head())

In [ ]:
# df_cast_expanded.to_csv('C:/Users/Usuario/Desktop/Data Science/SoyHenry/PI1/df_cast_expanded.csv', index=False)

In [ ]:
# 2) Tratamiento de valores nulos:

# Rellenamos los valores nulos de las columnas 'revenue' y 'budget' con 0
df_movies['REVENUE'].fillna(0, inplace=True)
df_movies['BUDGET'].fillna(0, inplace=True)

# Eliminamos las filas donde 'release_date' es nulo
df_movies = df_movies.dropna(subset=['RELEASE_DATE'])

# 3) Tratamiento de fechas:

# Aseguramos que la columna release_date esté en formato datetime
df_movies['RELEASE_DATE'] = pd.to_datetime(df_movies['RELEASE_DATE'], errors='coerce')

# Convertimos la fecha al formato AAAA-mm-dd
df_movies['RELEASE_DATE'] = df_movies['RELEASE_DATE'].dt.strftime('%Y-%m-%d')

# 4) Creación de columnas nuevas:

# Creamos una nueva columna release_year extrayendo el año de release_date
df_movies['RELEASE_YEAR'] = pd.to_datetime(df_movies['RELEASE_DATE']).dt.year

# Convertimos las columnas 'REVENUE' y 'BUDGET' a numéricas, forzando errores a NaN
df_movies['REVENUE'] = pd.to_numeric(df_movies['REVENUE'], errors='coerce')
df_movies['BUDGET'] = pd.to_numeric(df_movies['BUDGET'], errors='coerce')

# Creamos la columna 'RETURN' calculando REVENUE / BUDGET
df_movies['RETURN'] = df_movies.apply(lambda x: x['REVENUE'] / x['BUDGET'] if x['BUDGET'] > 0 else 0, axis=1)

# 5) Eliminación de columnas:

# Eliminamos las columnas video, imdb_id, adult, original_title, poster_path y homepage.
df_movies.drop(columns=['video', 'imdb_id', 'adult', 'original_title', 'poster_path', 'homepage'], inplace=True)

